In [3]:
!docker run --name mongodb -d -p 27017:27017 mongo:latest

3cbc12f71c20fd0cb59225ab9bc4a59a2c337c7d5959642969a3c904f9ef8379


In [2]:
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [4]:
%pip install llama-index-storage-docstore-mongodb
%pip install llama-index-storage-index-store-mongodb
%pip install llama-index


  Using cached llama_index_core-0.12.2-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.12.2-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.11.23
    Uninstalling llama-index-core-0.11.23:
      Successfully uninstalled llama-index-core-0.11.23
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index 0.11.23 requires llama-index-core<0.12.0,>=0.11.23, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-agent-openai 0.3.4 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-cli 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-embeddings-ollama 0.3.1 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-embeddings-openai 0.2.5 requires llama-index-core<0.12.0,>=0.11.0, but you have llama-index-core 0.12.2 which is incompatible.
llama-index-indices-managed-llama-cloud 0.4.0 requires llama-index-core<0.12.0,>=0.11.13.post1, but you have llama-index-core 0.12.2 which is incompatible.
llama-in

Note: you may need to restart the kernel to use updated packages.
  Using cached llama_index_core-0.11.23-py3-none-any.whl.metadata (2.5 kB)
Using cached llama_index_core-0.11.23-py3-none-any.whl (1.6 MB)
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.12.2
    Uninstalling llama-index-core-0.12.2:
      Successfully uninstalled llama-index-core-0.12.2
Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llama-index-embeddings-huggingface 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-extractors-entity 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-database 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-readers-deeplake 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-storage-docstore-mongodb 0.3.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-core 0.11.23 which is incompatible.
llama-index-storage-index-store-mongodb 0.4.0 requires llama-index-core<0.13.0,>=0.12.0, but you have llama-index-c

In [5]:
import os
import logging
from llama_index.core import SimpleDirectoryReader, StorageContext
from llama_index.core import VectorStoreIndex, SimpleKeywordTableIndex, SummaryIndex
from llama_index.llms.openai import OpenAI
from llama_index.core.node_parser import SentenceSplitter
from llama_index.storage.docstore.mongodb import MongoDocumentStore
from llama_index.storage.index_store.mongodb import MongoIndexStore


In [6]:

# Configure logging
logging.basicConfig(level=logging.INFO)

# MongoDB connection URI
MONGO_URI = "mongodb://localhost:27017"


In [7]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()
# documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()


In [8]:

# Step 2: Parse documents into nodes
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)
# Create a storage context

# Step 2: Create MongoDB Storage Context
docstore = MongoDocumentStore.from_uri(uri=MONGO_URI)
index_store = MongoIndexStore.from_uri(uri=MONGO_URI)

docstore.add_documents(nodes)  # Add nodes to the document store


storage_context = StorageContext.from_defaults(docstore=docstore, index_store=index_store)


In [9]:
#os.environ["OPENAI_API_KEY"] = getpass.getpass("open ai api key: ")
from llama_index.core import Settings
from llama_index.llms.ollama import  Ollama
Settings.llm = Ollama(model='llama3.2:latest', base_url='http://localhost:11434',temperature=0.1)



In [10]:
from llama_index.embeddings.ollama import OllamaEmbedding

ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",  # Replace with your desired model
    base_url="http://localhost:11434",  # Ensure Ollama is running at this endpoint
    ollama_additional_kwargs={"mirostat": 0} #Mirostat is a technique for controlling perplexity and balancing the text generation process in large language models (LLMs).
) 

In [11]:


# Step 4: Build Multiple Indexes
summary_index = SummaryIndex(nodes, storage_context=storage_context)
vector_index = VectorStoreIndex(nodes, storage_context=storage_context, embed_model=ollama_embedding)
keyword_table_index = SimpleKeywordTableIndex(nodes, storage_context=storage_context)


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/a

In [12]:
# Save index IDs for reloading later
summary_id = summary_index.index_id
vector_id = vector_index.index_id
keyword_id = keyword_table_index.index_id

# Print saved IDs (optional for verification)
print(f"Summary Index ID: {summary_id}")
print(f"Vector Index ID: {vector_id}")
print(f"Keyword Table Index ID: {keyword_id}")

Summary Index ID: e7dcc879-e5f6-4930-91b8-7631a4600e31
Vector Index ID: 5076eef0-36c8-4f70-a0b9-aad62f09081a
Keyword Table Index ID: 743e88ed-27b6-438a-b08f-f519a2f9226f


In [13]:
# Persist storage context (only vector store if needed)
storage_context.persist()


In [14]:
 #Save index IDs
index_ids = {
    "summary_id": summary_index.index_id,
    "vector_id": vector_index.index_id,
    "keyword_id": keyword_table_index.index_id,
}

# Save the index IDs for later use
import json
with open("mongodb_index_ids.json", "w") as f:
    json.dump(index_ids, f)
print("Index IDs saved.")

Index IDs saved.


In [17]:

# Verify that nodes are persisted
print(f"Total nodes in MongoDB DocumentStore: {len(docstore.docs)}")

# Step 5: Query the Indexes
# Configure OpenAI GPT model
#chatgpt = OpenAI(temperature=0, model="gpt-3.5-turbo")
#from llama_index.core import Settings

#Settings.llm = chatgpt

# Query summary index
query_engine = summary_index.as_query_engine()
summary_response = query_engine.query("What is a summary of this document?")
print("Summary Response:", summary_response)

# Query vector index
query_engine = vector_index.as_query_engine()
vector_response = query_engine.query("What did the author do growing up?")
print("Vector Response:", vector_response)

# Query keyword table index
query_engine = keyword_table_index.as_query_engine()
keyword_response = query_engine.query("What did the author do after his time at YC?")
print("Keyword Table Response:", keyword_response)


Total nodes in MongoDB DocumentStore: 21


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://loc

Summary Response: The document is a collection of personal anecdotes from an entrepreneur's experience at Y Combinator, a startup accelerator. The writer reflects on various challenges they faced, including disagreements with cofounders, dealing with difficult people, and navigating the complexities of online publishing. Through their experiences, they offer insights into the world of entrepreneurship, emphasizing the importance of adaptability, prioritization, and self-awareness.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:llama_index.core.indices.keyword_table.retrievers:> Starting query: What did the author do after his time at YC?
INFO:llama_index.core.indices.keyword_table.retrievers:query keywords: ['yc', 'time', 'author']
INFO:llama_index.core.indices.keyword_table.retrievers:> Extracted keywords: ['yc', 'time']


Vector Response: The author worked on writing short stories and programming as a hobby before college. They started with early versions of Fortran and later moved to microcomputers, where they began serious programming.


INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST http://localhost:11434/api/chat "HTTP/1.1 200 OK"


Keyword Table Response: The author continued to work on various projects after leaving Y Combinator. He spent several years writing about topics he had stacked up, including an essay on how to choose what to work on. He also started thinking about other things he could work on and began exploring new ideas.


In [18]:
from llama_index.core.response.notebook_utils import display_response

display_response(summary_response)
display_response(vector_response)
display_response(keyword_response)

**`Final Response:`** The document is a collection of personal anecdotes from an entrepreneur's experience at Y Combinator, a startup accelerator. The writer reflects on various challenges they faced, including disagreements with cofounders, dealing with difficult people, and navigating the complexities of online publishing. Through their experiences, they offer insights into the world of entrepreneurship, emphasizing the importance of adaptability, prioritization, and self-awareness.

**`Final Response:`** The author worked on writing short stories and programming as a hobby before college. They started with early versions of Fortran and later moved to microcomputers, where they began serious programming.

**`Final Response:`** The author continued to work on various projects after leaving Y Combinator. He spent several years writing about topics he had stacked up, including an essay on how to choose what to work on. He also started thinking about other things he could work on and began exploring new ideas.